In [14]:
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, IntSlider, Label
import cv2
import numpy as np
import io
from IPython.display import display, clear_output
import fileupload
from matplotlib import pyplot as plt
import matplotlib

# opcja inline powoduje, że obrazek wyświetla się po każdej zmianie parametrów
%matplotlib inline

# domyślne wartości parametrów
face_scale_factor = 1.05
face_neighbors = 4
eyes_scale_factor = 1.3
eyes_neighbors = 4

src_img = None

# zwraca obrazek z zaznaczonymi wykrytymi twarzami
def find_face(original_img, img, scale_factor, min_neighbors):
    # pobieranie klasyfikatora
    face_cascade = cv2.CascadeClassifier('cascades/haarcascade_frontalface_alt.xml')
    # wykrywanie
    faces = face_cascade.detectMultiScale(original_img, scale_factor, min_neighbors)
    
    # zaznacza wykryte twarze
    for (x,y,w,h) in faces:
        # rysuje prostokąty
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,215,0),5)

    return img
    
# zwraca obrazek z zaznaczonymi wykrytymi oczami
def find_eyes(original_img, img, scale_factor, min_neighbors):
    # pobieranie klasyfikatora
    eyes_cascade = cv2.CascadeClassifier('cascades/haarcascade_eye.xml')
    # wykrywanie
    eyes = eyes_cascade.detectMultiScale(original_img, scale_factor, min_neighbors)
    
    # zaznacza wykryte oczy
    for (x,y,w,h) in eyes:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),5)
        
    return img

# pokazuje obrazek
def show_img(img):
    # konwersja przestrzeni barw
    # niezbędna z powodu niezgodności kanałów poszczególnych składowych
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    # definicja rozmiaru obrazka
    plt.figure(figsize=(20,10))
    # wyświetlanie obrazka
    plt.imshow(img)

# przetwarza obrazek
def process_img(face_scale_factor, face_neighbors, eyes_scale_factor, eyes_neighbors):
    # sprawdza czy istnieje obrazek
    if src_img is not None:
        # tworzy kopię roboczą obrazka
        img = src_img.copy()
        # oryginalny obrazek, który jest przetwarzany
        original_img = src_img.copy()
        # użycie funkcji wykrywających
        img = find_eyes(original_img, img, eyes_scale_factor, eyes_neighbors)
        img = find_face(original_img, img, face_scale_factor, face_neighbors)
        # wyświetlanie obrazku
        show_img(img)

# wyświtla suwaki
def sliders():
    # wyświetla pełen opis suwaka
    style = {'description_width': 'initial'}
    # funkcja wywołuje się podczas interakcji z suwakiem i przekazuje argumenty potrzebne do działania process_img
    interact(process_img,
             # deklaracja suwaka
             face_scale_factor=FloatSlider(
                    value=face_scale_factor,
                    min=1.01,
                    max=1.5,
                    step=0.01,
                    description='Face Scale Factor:',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='.2f',
                    style=style
                    ),
             face_neighbors=IntSlider(
                    value=face_neighbors,
                    min=2,
                    max=9,
                    step=1,
                    description='Face Min Neighbors:',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='d',
                    style=style
                    ),
             eyes_scale_factor=FloatSlider(
                    value=eyes_scale_factor,
                    min=1.01,
                    max=1.5,
                    step=0.01,
                    description='Eyes Scale Factor:',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='.2f',
                    style=style
                    ),
             eyes_neighbors=IntSlider(
                    value=eyes_neighbors,
                    min=2,
                    max=9,
                    step=1,
                    description='Eyes Min Neighbors:',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='d',
                    style=style
                    )
            );
    

# odpowiada za wyświetlanie i zachowanie okienka wyboru plików
# https://github.com/peteut/ipython-file-upload
def upload():
    # napis
    label = Label(value=unicode('Wybierz z dysku zdjęcie, na którym chcesz pracować: ', 'utf-8'))
    display(label)
    # tworzenie widżetu
    _upload_widget = fileupload.FileUploadWidget()
    
    # konwersja base64 do tablicy np.array
    # a następnie dekodowanie do formatu czytelnego dla OpenCV
    # https://stackoverflow.com/a/42538142/1589989
    def data_uri_to_cv2_img(uri):
        encoded_data = uri.split(',')[1]
        numpy_array = np.fromstring(encoded_data.decode('base64'), np.uint8)
        img = cv2.imdecode(numpy_array, cv2.IMREAD_COLOR)
        return img
    
    # uruchamia się za każdym razem, gdy wybierany jest obrazek
    def callback(change):
        global src_img
        # kopiowanie obrazka w formacie base64 do zmiennej globalnej
        src_img = data_uri_to_cv2_img(change['owner'].data_base64)
        # czyszczenie wyjścia (ekranu)
        clear_output()
        # ponowne wyświetlenie widżetu wyboru obrazka i suwaków 
        upload()
        sliders()      
        
    # przypisanie funkcji do widżetu wyboru obrazka
    _upload_widget.observe(callback, names='data')
    # wyświetla widżet
    display(_upload_widget)

# wyświetlenie widżetu wyboru obrazka i suwaków 
upload()  
sliders()

# poniższy kod powinien uruchamiać wyświetlanie obrazku z kamery
# ale może nie działać

# cap = cv2.VideoCapture(0)
# while(True):
#     # Capture frame-by-frame
#     ret, frame = cap.read()
#     img = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
#     show_img(img)
